In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from. selenium.webdriver import ActionsChains as AC
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
import time

In [2]:
options =  webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')

driver = webdriver.Chrome("./chromedriver")

In [3]:
driver.get('https://www.atrapalo.com/hoteles')

In [4]:
driver.find_element_by_class_name("default-input.tt-input").click()


In [5]:
driver.find_element_by_class_name("default-input.tt-input").send_keys("valencia")

In [6]:
driver.find_element_by_class_name("boton-buscar.form-submit.columns.large-1.large-uncentered.show-for-large").click()

In [7]:
time.sleep(10)
driver.find_element_by_xpath('//*[@id="sortingContainer"]/nav/div[2]/ul/li[2]').click()

In [8]:
texto_columnas = driver.find_element_by_class_name('hotels-list.box-resultados')

## Sacar Hoteles

In [9]:
texto_hoteles = texto_columnas.find_elements_by_class_name('product-name')
hotel = texto_hoteles[0:3]
time.sleep(2)

In [10]:
time.sleep(2)
hoteles = []
for i in hotel:
    primero = i.get_attribute("title")
    hoteles.append(primero)

## Sacar Precio

In [11]:
texto_precio = texto_columnas.find_elements_by_class_name('value')
texto_precio = texto_precio[0:3]

In [12]:
precio = []
for i in texto_precio:
    precio.append(i.text)
    time.sleep(1)

## Sacar distancia

In [13]:
texto_distancia = texto_columnas.find_elements_by_class_name('item-location')
texto_distancia = texto_distancia[0:3]

In [14]:
distancia = []
for i in texto_distancia:
    distancia.append(i.text)

## Sacar dirección

In [15]:
direcc = []
for i in hotel:
    primero = i.get_attribute("href")
    direcc.append(primero)

In [16]:
direcc = direcc[0:4]
direcc

['https://www.atrapalo.com/hoteles/88951-0_russafa-youth-hostel?atoken=b742f9fa6a920a5b5b62ca09ef3710cb&utoken=1:10:0:0:4b0ed&noredirect=1',
 'https://www.atrapalo.com/hoteles/164081-0_hotel-travelodge-valencia-aeropuerto-manises-?atoken=b742f9fa6a920a5b5b62ca09ef3710cb&utoken=1:10:0:0:4b0ed&noredirect=1',
 'https://www.atrapalo.com/hoteles/730644-0_bed-i-breakfast-hi-valencia-canovas?atoken=b742f9fa6a920a5b5b62ca09ef3710cb&utoken=1:10:0:0:4b0ed&noredirect=1']

In [17]:
driver.quit()

In [18]:
time.sleep(5)
address = []
for link in direcc:
    driver = webdriver.Chrome("./chromedriver")
    driver.get(link)
    time.sleep(3)
    calle = driver.find_element_by_css_selector("span.app-address")
    calle = calle.text
    address.append(calle)
    driver.quit()

In [19]:
hoteles

['Russafa Youth Hostel Albergue',
 'Hotel Travelodge Valencia Aeropuerto (manises) 1 Estrella',
 'Bed & Breakfast Hi Valencia Cánovas Bed&Breakfast']

In [20]:
precio

['32€', '38€', '42€']

In [21]:
distancia

['Valencia a 0,57 km del centro',
 'Valencia a 8,46 km del centro',
 'Valencia a 0,92 km del centro']

In [22]:
address

['Padre Perera, 5 - 1º, Valencia 46006, Valencia (España) ',
 'C/ Las Rosas, 31 - 46940, Valencia 46940, Valencia (España) ',
 'Calle Cirilo Amorós 82 pta. 6, Valencia 46004, Valencia (España) ']